In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.sql import SparkSession
from pyspark.mllib.classification import LabeledPoint
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from time import perf_counter

In [2]:
conf = SparkConf().setMaster("spark://10.99.250.19:7077").setAppName("SUSY")
conf.set("spark.executor.memory", "20G")
conf.set("spark.driver.memory", "2G")
conf.set("spark.executor.cores", "8")
conf.set("spark.cores.max", "8")
# conf.set("spark.executor.instances", "2")

conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
# conf.set("spark.default.parallelism", "16")
# create a Spark Session instead of a Spark Context
spark = SparkSession.builder \
    .config(conf = conf) \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/05 13:24:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option("delimiter", ",").option("header", "false").csv('/work/li.baol/data/SUSY.csv')

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/li.baol/anaconda3/envs/pytorch1.10/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/li.baol/anaconda3/envs/pytorch1.10/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/li.baol/anaconda3/envs/pytorch1.10/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
data = df.rdd.map(lambda row: LabeledPoint(float(row['_c0']), list(map(lambda x: float(x), row.asDict().values()))[1:]))
# data.take(2)

In [ ]:
# (trainingData, testData) = data.randomSplit([0.7, 0.3])

In [ ]:
# testData.collect()

In [ ]:
start = perf_counter()
model = DecisionTree.trainClassifier(data, numClasses=2, categoricalFeaturesInfo={},
                                    impurity='gini', maxDepth=8)
end = perf_counter()
end-start                                    

36.77679096348584

In [ ]:
predictions = model.predict(data.map(lambda x: x.features))
predictions

In [ ]:
# labelsAndPredictions = trainingData.map(lambda lp: lp.label).zip(predictions)
# testErr = labelsAndPredictions.filter(
#     lambda lp: lp[0] == lp[1]).count() / float(trainingData.count())
# print('Test Error = ' + str(testErr))
